# Part IV - Decompounded Options

#### Question 1

A decompounded option pays the following at time T = 5y:  

CMS $10y^{1/p}$ - $0.04^{1/q}$

where p = 4 and q = 2. Use static replication to value the PV of this payoff.

$g(F) = F^{\frac{1}{4}} - 0.2 \ \ \ \ \ \ g'(F) = \frac{1}{4}F^{-\frac{3}{4}} \ \ \ \ \ \ \ g''(F) = -\frac{3}{16}F^{-\frac{7}{4}}$ 

$ h(K) = \frac{g(K)}{IRR(K)}$ <br><br>
$ h'(K) = \frac{IRR(K)g'(K) - g(K)IRR'(K)}{IRR(K)^2}$<br><br>
$ h''(K) = \frac{IRR(K)g''(K) - IRR''(K)g(K) - 2 * IRR'(K)g'(K)}{IRR(K)^2} +\frac{2*IRR'(K)^2g(K)}{IRR(K)^3}$

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import interpolate
from math import log, sqrt, exp
from scipy import integrate

#### Import data files

In [2]:
data_FS = pd.read_csv('df_ForwardSwap.csv', header = 0, index_col = 0)  
data_DF = pd.read_csv('df_comb.csv', header = 0, index_col = 0)  
data_SABR_Alpha=pd.read_csv('SABR_Alpha.csv', header = 0, index_col = 0)
data_SABR_Alpha=data_SABR_Alpha.drop(data_SABR_Alpha.index[[0,1]])
data_SABR_Alpha.columns=[1,2,3,5,10]
data_SABR_Nu=pd.read_csv('SABR_Nu.csv', header = 0, index_col = 0)
data_SABR_Nu=data_SABR_Nu.drop(data_SABR_Nu.index[[0,1]])
data_SABR_Nu.columns=[1,2,3,5,10]
data_SABR_Rho=pd.read_csv('SABR_Rho.csv', header = 0, index_col = 0)
data_SABR_Rho=data_SABR_Rho.drop(data_SABR_Rho.index[[0,1]])
data_SABR_Rho.columns=[1,2,3,5,10]

In [3]:
data_SABR_Rho

,1,2,3,5,10
1,-0.632628,-0.524618,-0.482388,-0.413979,-0.264429
5,-0.584666,-0.546609,-0.549566,-0.511825,-0.435195
10,-0.546160,-0.544090,-0.549840,-0.563144,-0.506740


#### Define Black and SABR functions

In [4]:
def Black76Lognormal(F, K, r, sigma, T, opt):
    d1 = (log(F/K)+(sigma*sigma/2)*T)/(sigma*sqrt(T))
    d2 = d1-sigma*sqrt(T)
    if opt == 'Call':
        return F*exp(-r*T)*norm.cdf(d1) - K*exp(-r*T)*norm.cdf(d2)
    elif opt == 'Put':
        return K*exp(-r*T)*norm.cdf(-d2) - F*exp(-r*T)*norm.cdf(-d1)

def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*log(F/X)
        zhi = log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

#### Define IRR Functions

Let $m$ denote the payment frequenc ($m=2$ for semi-annual payment frequency), and let $N$ denote the tenor of the swap (number of years), the partial derivatives on the IRR function $\mbox{IRR}(S)$ given by:
\begin{equation*}
\begin{split}
\mbox{IRR}(K)&=\sum_{i=1}^{N\times m}\frac{1}{(1+\frac{K}{m})^i}=\frac{1}{K}\left[1-\frac{1}{\left(1+\frac{K}{m}\right)^{N\times m}}\right]\\
\mbox{IRR}'(K)&=-\frac{1}{K}\mbox{IRR}(K)
+\frac{1}{m\times K}\frac{N\times m}{\left(1+\frac{K}{m}\right)^{N\times m+1}} \\
\mbox{IRR}''(K)&=-\frac{2}{K}\mbox{IRR}'(K)
-\frac{1}{m^2\times K}\frac{N\times m\cdot (N\times m+1)}{\left(1+\frac{K}{m}\right)^{N\times m+2}} \\
\end{split}
\end{equation*}

These results will need to be generalised to handle the case for $m=2$ to be consistent with the semi-annual payment frequency swap market data provided.

In [5]:
def IRR_0(K, m, N):
    # implementation of IRR(K) function
    value = 1/K * ( 1.0 - 1/(1 + K/m)**(N*m) )
    return value

def IRR_1(K, m, N):
    # implementation of IRR'(K) function (1st derivative)
    firstDerivative = -1/K*IRR_0(K, m, N) + 1/(K*m)*N*m/(1+K/m)**(N*m+1)
    return firstDerivative

def IRR_2(K, m, N):
    # implementation of IRR''(K) function (2nd derivative)
    secondDerivative = -2/K*IRR_1(K, m, N) - 1/(K*m*m)*(N*m)*(N*m+1)/(1+K/m)**(N*m+2)
    return secondDerivative

In [6]:
#define payoff functions g(x) and h(x)
def g_0(K):
    return K**(1/4) - 0.2

def g_1(K):
    return (1/4)*(K)**(-3/4)

def g_2(K):
    return (-3/16)*(K)**(-7/4)

def h_0(K, m, N):
    # implementation of h(K)
    value = g_0(K) / IRR_0(K, m, N)
    return value

def h_1(K, m, N):
    # implementation of h'(K) (1st derivative)
    firstDerivative = (IRR_0(K, m, N)*g_1(K) - g_0(K)*IRR_1(K, m, N)) / IRR_0(K, m, N)**2
    return firstDerivative

def h_2(K, m, N):
    # implementation of h''(K) (2nd derivative)
    secondDerivative = ((IRR_0(K, m, N)*g_2(K) - IRR_2(K, m, N)*g_0(K) - 2.0*IRR_1(K, m, N)*g_1(K))/IRR_0(K, m, N)**2 
                        + 2.0*IRR_1(K, m, N)**2*g_0(K)/IRR_0(K, m, N)**3)
    return secondDerivative

$ V_0  = D(0,T)g(F) + \int_0^F h''(K)V^{rec}(K)dK + \int_F^\infty h''(K)V^{pay}(K)dK $

In [7]:
F = data_FS.loc[9, 'Swap_Rate']   # this is 5y x 10y forward swap rate
D = data_DF.loc[9, 'OIS Discount Factor']     # this is OIS discount factor D(0, 5y)

alpha = data_SABR_Alpha.iloc[1, 4] # this is alpha parameter for 5y x 10y forward swap
beta = 0.9                        # this is pre-determined beta at 0.9
rho = data_SABR_Rho.iloc[1, 4]    # this is rho parameter for 5y x 10y forward swap
nu = data_SABR_Nu.iloc[1, 4]     # this is nu parameter for 5y x 10y forward swap


M = 2
N = 10
T = 5

V_rec = integrate.quad(lambda x: h_2(x, M, N)*Black76Lognormal(F, x, 0, SABR(F, x, T, alpha, 0.9, rho, nu),T, "Put"),
                                                    0,F)
V_pay = integrate.quad(lambda x: h_2(x, M, N)*Black76Lognormal(F, x, 0, SABR(F, x, T, alpha, 0.9, rho, nu),T, "Call"),
                                                    F,1000)
                                                   
PVoption = D * g_0(F) + V_rec[0] + V_pay[0]

/tmp/ipykernel_36200/2891964737.py:14: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  V_rec = integrate.quad(lambda x: h_2(x, M, N)*Black76Lognormal(F, x, 0, SABR(F, x, T, alpha, 0.9, rho, nu),T, "Put"),


In [8]:
print ("The PV of this option using static replication is:",PVoption )

The PV of this option using static replication is: 0.2495412082099216


#### Question 2

(2) Supose the payoff is now  

(CMS $10y^{1/p}$ - $0.04^{1/q}$)$^+$

Use static replication to value the PV of this payoff.

$F^\frac{1}{4} > 0.2$<br>
$F > 0.2^4$<br>
$F > 0.0016 = L$
$CMS\ Caplet = h'(L)V^{pay}(L) + \int^{\infty}_Lh''(K)V^{pay}(K)dK$

In [9]:
Tenor = 10
Delta = 0.5
T = 5
L = 0.2**4

M = 2
N = 10

term1 = h_1(L, M, N)*Black76Lognormal(F, L, 0, SABR(F, L, T, alpha, 0.9, rho, nu),T,"Call")
term2 = integrate.quad(lambda x: h_2(x, M, N)*Black76Lognormal(F, x, 0, SABR(F, x, T, alpha, 0.9, rho, nu),T,"Call"),
                                                    L,1000)
PV_caplet = term1+ term2[0]
PV_caplet

0.030989894458155975

In [10]:
print ("The PV of this option using static replication is:",PV_caplet )

The PV of this option using static replication is: 0.030989894458155975
